# Multilabel-multiclass

In [ ]:
# For dataset.
from sklearn.datasets import make_multilabel_classification
from qiskit_algorithms.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [ ]:
# for data split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap

In [ ]:
X, y = make_multilabel_classification(
    n_samples=10000,
    n_features=2,
    n_classes=2,
    n_labels=2,
    # n_redundant=0,
    # n_clusters_per_class=1,
    # class_sep=2.0,
    random_state=algorithm_globals.random_seed,
)

In [ ]:
X.shape, y.shape

In [ ]:
y[:10] # = [3,1,2,0,2,3,3,3,3,3 ...]

In [ ]:
## Train test split
train_features, test_features, train_labels, test_labels = train_test_split(
    X, y, train_size=0.8, random_state=algorithm_globals.random_seed
)

In [ ]:
train_labels_new = train_labels[:,0]*1+train_labels[:,1]*2

In [ ]:
test_labels_new = test_labels[:,0]*1+test_labels[:,1]*2

In [ ]:
train_labels_new[:10]

In [ ]:
num_features = X.shape[1]

In [ ]:
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=3)
feature_map.decompose().draw(output="mpl", fold=20)

In [ ]:
from qiskit.circuit.library import RealAmplitudes

ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
# ansatz.decompose().draw(output="mpl", fold=20)

In [ ]:
# defining the optimiser
optimizer = COBYLA(maxiter=100)

In [ ]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)


def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
vqc = VQC(
    # sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    loss='cross_entropy',
    optimizer=optimizer,
    callback=callback_graph,
)

# clear objective value history
objective_func_vals = []

start = time.time()
vqc.fit(train_features, train_labels_new)
elapsed = time.time() - start

print(f"Training time: {round(elapsed)} seconds")

In [ ]:
# testing the QSVC scores
train_score_q4 = vqc.score(train_features, train_labels_new)
test_score_q4 = vqc.score(test_features, test_labels_new)

print(f"Quantum VQC on the training dataset: {train_score_q4:.2f}")
print(f"Quantum VQC on the test dataset:     {test_score_q4:.2f}")

In [ ]:
predictions = vqc.predict(test_features)

In [ ]:
predictions[:20]

In [ ]:
test_labels_new[:20]

In [ ]:
predictions.shape

In [ ]:
# Define the mapping
mapping = {0: [0, 0], 1: [1, 0], 2: [0, 1], 3: [1, 1]}

# Use list comprehension to create the new array
predictions_labels_final = np.array([mapping[val] for val in predictions])

print(predictions_labels_final[:5])

In [ ]:
predictions_labels_final.shape

In [ ]:
test_labels.shape

In [ ]:
accuracy_score(predictions_labels_final, test_labels)